# B站流行动漫影评词云分析

In [85]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'F:\陶志远\数据\数据集\B站流行动漫影评数据\B站流行动漫影评数据.csv')
df

,author,score,disliked,likes,liked,ctime,content,last_ep_index,cursor,date
0,緖山まひろ,10,0,9981,0,1530982477,我有个想法，动物体内的细胞会不会是兽耳呢d(ŐдŐ๑),0,77395314810369,2018-07-07 16:54:37
1,onlylove丶亚丝娜,10,0,1,0,1557139155,题材新颖,0,77395314810369,2019-05-06 10:39:15
2,AKAHANE_铃木羽,10,0,3,0,1556947242,坐等第二季！！表白白细胞∠( ᐛ 」∠)＿,0,77395314810369,2019-05-04 05:20:42
3,妖怪退散,10,0,0,0,1557182714,好看，推荐,0,77395314810369,2019-05-06 22:45:14
4,受虎大王,10,0,0,0,1557118709,涨知识,0,77395314810369,2019-05-06 04:58:29
...,...,...,...,...,...,...,...,...,...,...
19058,八奇大蛇,2,0,1,0,1533969368,不配,0,74186973181452,2018-08-11 06:36:08
19059,我不听我就不改名,2,0,0,0,1533473635,第一次对一个网站以及一部番恶心到这种地步,0,74186973181452,2018-08-05 12:53:55
19060,截教至圣通天教主,2,0,0,0,1530984474,还可以,0,71618582349408,2018-07-07 17:27:54
19061,小寒、很忙,8,0,0,0,1530983251,血小板太萌了,0,71618582349408,2018-07-07 17:07:31


In [86]:
'''删除不需要的数据'''

del_col = ['disliked','likes','liked','ctime','last_ep_index','cursor','date']   # 要删除的列
data = df.drop(del_col, axis=1)

In [87]:
data

,author,score,content
0,緖山まひろ,10,我有个想法，动物体内的细胞会不会是兽耳呢d(ŐдŐ๑)
1,onlylove丶亚丝娜,10,题材新颖
2,AKAHANE_铃木羽,10,坐等第二季！！表白白细胞∠( ᐛ 」∠)＿
3,妖怪退散,10,好看，推荐
4,受虎大王,10,涨知识
...,...,...,...
19058,八奇大蛇,2,不配
19059,我不听我就不改名,2,第一次对一个网站以及一部番恶心到这种地步
19060,截教至圣通天教主,2,还可以
19061,小寒、很忙,8,血小板太萌了


In [88]:
'''查看是否含有缺失值'''
data.isnull().sum()

author     0
score      0
content    0
dtype: int64

In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
%matplotlib inline

In [91]:
data['senti'] = data['score'] >= 8
data['senti'] = data['senti'].replace([True, False], ['pos', 'neg'])  # replace(x, y),表示y代替x
data

,author,score,content,senti
0,緖山まひろ,10,我有个想法，动物体内的细胞会不会是兽耳呢d(ŐдŐ๑),pos
1,onlylove丶亚丝娜,10,题材新颖,pos
2,AKAHANE_铃木羽,10,坐等第二季！！表白白细胞∠( ᐛ 」∠)＿,pos
3,妖怪退散,10,好看，推荐,pos
4,受虎大王,10,涨知识,pos
...,...,...,...,...
19058,八奇大蛇,2,不配,neg
19059,我不听我就不改名,2,第一次对一个网站以及一部番恶心到这种地步,neg
19060,截教至圣通天教主,2,还可以,neg
19061,小寒、很忙,8,血小板太萌了,pos


In [96]:
'''
短句过滤
'''
data = data[data.iloc[:, 2].apply(len) >= 5]
data

,author,score,content,senti
0,緖山まひろ,10,我有个想法，动物体内的细胞会不会是兽耳呢d(ŐдŐ๑),pos
2,AKAHANE_铃木羽,10,坐等第二季！！表白白细胞∠( ᐛ 」∠)＿,pos
3,妖怪退散,10,好看，推荐,pos
5,镰刀灬锤子,10,以B站的标准，工作细胞完全是9.9分啊，为什么才9.7啊？是有孤儿故意拉低分吗？,pos
7,我还是周一到周六,10,真的太可爱了呜呜呜，声优表惊艳到爆炸！,pos
...,...,...,...,...
19055,袖中火,2,没法当老婆差评,neg
19056,无限回轮弑魂者,2,没法看了开始水,neg
19059,我不听我就不改名,2,第一次对一个网站以及一部番恶心到这种地步,neg
19061,小寒、很忙,8,血小板太萌了,pos


In [99]:
'''
情感分析
'''
from snownlp import SnowNLP # 情感分析语言处理库
# 语义积极的概率，越接近 1 情感表现越积极
coms = data['content'].apply(lambda x: SnowNLP(x).sentiments)
print('情感分析后')
positive_data = data[coms >= 0.9]
negative_data = data[coms <= 0.1]

情感分析后


In [102]:
print('特别喜欢的')
positive_data

特别喜欢的


,author,score,content,senti
7,我还是周一到周六,10,真的太可爱了呜呜呜，声优表惊艳到爆炸！,pos
9,我是锡旺,10,虽然番剧涉及的生物知识不是完全正确，但是很好看，也发人深省。,pos
10,萨_森,10,当你觉得孤独无助时 想一想还有十几亿的细胞 只为了你一个人而活,pos
11,幼稚小孩长大了,10,血小板让我明白萌即正义的含义！,pos
12,清纯无邪の少年,10,题材新颖 剧情也不错 作画也很好 最重要的是声优阵容豪华,pos
...,...,...,...,...
19016,染瑾年ii,2,一般，OP不好听，单纯为了一个血小板的就给这么高的评分是不理智的，有点创意但是感觉画风有点单...,neg
19017,Aircraft-carrier,2,过于血腥暴力和恐怖，不适合未成年人观看，建议下架。,neg
19029,Tea_Drinker,2,全篇最有趣的地方就在于作者将细胞和疾病的运作方式以一种科学且有趣的形式演绎出来，并在此过程中...,neg
19032,人形何首污,2,七月霸权是属于骨王的！,neg


In [103]:
print('不喜欢的')
negative_data

不喜欢的


,author,score,content,senti
59,这只猪我不要了,10,来晚的评分，请收下！,pos
82,菜鸟渣渣,10,来晚的评分，请收下！,pos
99,Love灬逗比,10,全站霸榜第一番，居然9.7分？什么情况？,pos
122,少少狈,10,想看杀手t细胞和辅助t细胞的本子,pos
137,三玖二十一く,10,承包血小板,pos
...,...,...,...,...
19046,孤单橙子,2,说实话真没那么好看，看得挺尴尬的，网上有些吹过了啊，都是来看血小板的吗，虽说新番应该支持，和...,neg
19051,夜雨風花,2,不说了卡成狗,neg
19055,袖中火,2,没法当老婆差评,neg
19056,无限回轮弑魂者,2,没法看了开始水,neg


In [104]:
'''
分词分析
'''
import jieba
my_cut = lambda s: ' '.join(jieba.cut(s)) # 自定义简单分词函数
positive_ser = positive_data.iloc[:, 2].apply(my_cut) # 通过“广播机制”分词，加快速度
negative_ser = negative_data.iloc[:, 2].apply(my_cut)

In [108]:
print('大于 0.5---正面数据---分词')
pd.DataFrame(positive_ser)

大于 0.5---正面数据---分词


,content
7,真的 太 可爱 了 呜呜 呜 ， 声优 表 惊艳 到 爆炸 ！
9,虽然 番剧 涉及 的 生物 知识 不是 完全正确 ， 但是 很 好看 ， 也 发人深省 。
10,当 你 觉得 孤独 无助 时 想一想 还有 十几亿 的 细胞 只 为了 你 一个 人而活
11,血小板 让 我 明白 萌即 正义 的 含义 ！
12,题材 新颖 剧情 也 不错 作画 也 很 好 ...
...,...
19016,一般 ， OP 不好 听 ， 单纯 为了 一个 血小板 的 就 给 这么 高 的 评分 是 ...
19017,过于 血腥 暴力 和 恐怖 ， 不 适合 未成年人 观看 ， 建议 下架 。
19029,全篇 最 有趣 的 地方 就 在于 作者 将 细胞 和 疾病 的 运作 方式 以 一种 科学...
19032,七月 霸权 是 属于 骨王 的 ！


In [109]:
print('小于 0.5---负面数据---分词')
pd.DataFrame(negative_ser)

小于 0.5---负面数据---分词


,content
59,来晚 的 评分 ， 请 收下 ！
82,来晚 的 评分 ， 请 收下 ！
99,全站 霸榜 第 一番 ， 居然 9.7 分 ？ 什么 情况 ？
122,想 看 杀手 t 细胞 和 辅助 t 细胞 的 本子
137,承包 血小板
...,...
19046,说实话 真 没 那么 好看 ， 看得 挺 尴尬 的 ， 网上 有些 吹过 了 啊 ， 都 是...
19051,不 说 了 卡成 狗
19055,没法 当 老婆 差评
19056,没法 看 了 开始 水


In [162]:
stops = pd.read_csv('F:\陶志远\Jupyter Notebook\AIBD重新体验\对应数据集\自然语言处理（2）——关键词提取\stopwords.txt')
# sep 设置分割词，由于 csv 默认以半角逗号为分割此，而该词恰好在停用词表中，因此会导致读取出错
# 所以解决办法是手动设置一个不存在的分割词，如 tipdm；
stops = [' ',''] + list(stops.values[:,0]) # pandas 自动过滤了空格符，这里手动添加
positive_df = pd.DataFrame(positive_ser)
negative_df = pd.DataFrame(negative_ser)

positive_df[1] = positive_df.iloc[:,0].apply(lambda s: s.split(' ')) # 定义一个分割函数
#positive_df[2] = positive_df.iloc[:,1].apply(lambda x: [i for i in x if i.encode('utf-8') not in stops])
negative_df[1] = negative_df.iloc[:,0].apply(lambda s: s.split(' ')) # 定义一个分割函数
#negative_df[2] = negative_df.iloc[:,1].apply(lambda x: [i for i in x if i.encode('utf-8') not in stops])

In [163]:
print('去停用词后：positive_df')
positive_df

去停用词后：positive_df


,content,1
7,真的 太 可爱 了 呜呜 呜 ， 声优 表 惊艳 到 爆炸 ！,"[真的, 太, 可爱, 了, 呜呜, 呜, ，, 声优, 表, 惊艳, 到, 爆炸, ！]"
9,虽然 番剧 涉及 的 生物 知识 不是 完全正确 ， 但是 很 好看 ， 也 发人深省 。,"[虽然, 番剧, 涉及, 的, 生物, 知识, 不是, 完全正确, ，, 但是, 很, 好看..."
10,当 你 觉得 孤独 无助 时 想一想 还有 十几亿 的 细胞 只 为了 你 一个 人而活,"[当, 你, 觉得, 孤独, 无助, 时, , , 想一想, 还有, 十几亿, 的, 细胞,..."
11,血小板 让 我 明白 萌即 正义 的 含义 ！,"[血小板, 让, 我, 明白, 萌即, 正义, 的, 含义, ！]"
12,题材 新颖 剧情 也 不错 作画 也 很 好 ...,"[题材, 新颖, , , , , , , , , 剧情, 也, 不错, , , , , , ..."
...,...,...
19016,一般 ， OP 不好 听 ， 单纯 为了 一个 血小板 的 就 给 这么 高 的 评分 是 ...,"[一般, ，, OP, 不好, 听, ，, 单纯, 为了, 一个, 血小板, 的, 就, 给..."
19017,过于 血腥 暴力 和 恐怖 ， 不 适合 未成年人 观看 ， 建议 下架 。,"[过于, 血腥, 暴力, 和, 恐怖, ，, 不, 适合, 未成年人, 观看, ，, 建议,..."
19029,全篇 最 有趣 的 地方 就 在于 作者 将 细胞 和 疾病 的 运作 方式 以 一种 科学...,"[全篇, 最, 有趣, 的, 地方, 就, 在于, 作者, 将, 细胞, 和, 疾病, 的,..."
19032,七月 霸权 是 属于 骨王 的 ！,"[七月, 霸权, 是, 属于, 骨王, 的, ！]"


In [164]:
print('去停用词后：negative_df')
negative_df

去停用词后：negative_df


,content,1
59,来晚 的 评分 ， 请 收下 ！,"[来晚, 的, 评分, ，, 请, 收下, ！]"
82,来晚 的 评分 ， 请 收下 ！,"[来晚, 的, 评分, ，, 请, 收下, ！]"
99,全站 霸榜 第 一番 ， 居然 9.7 分 ？ 什么 情况 ？,"[全站, 霸榜, 第, 一番, ，, 居然, 9.7, 分, ？, 什么, 情况, ？]"
122,想 看 杀手 t 细胞 和 辅助 t 细胞 的 本子,"[想, 看, 杀手, t, 细胞, 和, 辅助, t, 细胞, 的, 本子]"
137,承包 血小板,"[承包, 血小板]"
...,...,...
19046,说实话 真 没 那么 好看 ， 看得 挺 尴尬 的 ， 网上 有些 吹过 了 啊 ， 都 是...,"[说实话, 真, 没, 那么, 好看, ，, 看得, 挺, 尴尬, 的, ，, 网上, 有些..."
19051,不 说 了 卡成 狗,"[不, 说, 了, 卡成, 狗]"
19055,没法 当 老婆 差评,"[没法, 当, 老婆, 差评]"
19056,没法 看 了 开始 水,"[没法, 看, 了, 开始, 水]"


In [160]:
from gensim import corpora, models
corpora.Dictionary(positive_df[2])

In [161]:
from gensim import corpora, models
# 正面主题分析
pos_dict = corpora.Dictionary(positive_df[2]) # 建立词典
pos_corpus = [pos_dict.doc2bow(i) for i in positive_df[2]] # 建立语料库
pos_lda = models.LdaModel(pos_corpus, num_topics=3, id2word=pos_dict) # LDA 模型训练
print('#正面主题分析')
for i in range(3):
    print('topic', i)
    print(pos_lda.print_topic(i)) # 输出每个主题
# 负面主题分析
neg_dict = corpora.Dictionary(negative_df[2])
neg_corpus = [neg_dict.doc2bow(i) for i in negative_df[2]] # 建立语料库
neg_lda = models.LdaModel(neg_corpus, num_topics=3, id2word=neg_dict) # LDA 模型训练
print('#负面主题分析')
for i in range(3):
    print('topic', i)
    print(neg_lda.print_topic(i)) # 输出每个主题

#正面主题分析
topic 0
0.152*"" + 0.024*"的" + 0.021*")" + 0.021*"(" + 0.019*"血小板" + 0.017*"，" + 0.015*"了" + 0.015*"可爱" + 0.013*"我" + 0.012*"好看"
topic 1
0.091*"，" + 0.065*"的" + 0.024*"。" + 0.022*"了" + 0.019*"很" + 0.018*"我" + 0.014*"番" + 0.013*"！" + 0.012*"科普" + 0.011*"是"
topic 2
0.079*"！" + 0.038*"的" + 0.033*"，" + 0.019*"啊啊啊" + 0.015*"血小板" + 0.015*"了" + 0.013*"番" + 0.013*"啊" + 0.012*"科普" + 0.011*"可爱"
#负面主题分析
topic 0
0.071*"！" + 0.039*"，" + 0.031*"。" + 0.027*"的" + 0.024*"我" + 0.018*"了" + 0.012*"血小板" + 0.009*"吹" + 0.008*"是" + 0.008*"啊"
topic 1
0.083*"" + 0.039*"，" + 0.028*"我" + 0.028*"的" + 0.023*"了" + 0.013*"是" + 0.012*"吹" + 0.012*"爆" + 0.011*"就" + 0.009*"。"
topic 2
0.129*"！" + 0.049*"，" + 0.047*"血小板" + 0.023*"了" + 0.022*"的" + 0.018*"我" + 0.017*"承包" + 0.013*"" + 0.011*"。" + 0.008*"？"
